In [1]:
using IntervalArithmetic,IntervalBoxes,IntervalArithmetic.Symbols,Plots

In [2]:
x1 = 1 / 4
x2 = 2 / 4
x3 = 3 / 4
h = 1
g = 9.8
function calculate_expression(y1::Interval{T}, y2::Interval{T}, y3::Interval{T}) where T
    term1 = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1))
    term2 = sqrt((x1 - x2)^2 + (y1 - y2)^2) / (sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2)))
    term3 = sqrt((y2 - y3)^2 + (x2 - x3)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * (h - y3)))
    term4 = sqrt(y3^2 + (x3 - h)^2) / (sqrt(2 * g * (h - y3)) + sqrt(2 * g * h))
    return 2*(term1 + term2 + term3 + term4)
end

calculate_expression (generic function with 1 method)

In [3]:

# Define the branch-and-bound function
function calculate_branch_bound(f, Y1::Interval{T}, Y2::Interval{T}, Y3::Interval{T}, N) where T
    interval_lists = [[Y1, Y2, Y3]]
    upper_bound = sup(f(Y1..., Y2..., Y3...))
    upper_bounds = [upper_bound]
    working_list = [(Y1..., Y2..., Y3...)]

    for i in 1:N
        if isempty(working_list)
            break
        end
        
        # Pop the first interval pair
        (current_Y1, current_Y2, current_Y3) = popfirst!(working_list)

        # Update the upper bound
        upper_bound = min(upper_bound, sup(f(interval(mid(current_Y1))..., interval(mid(current_Y2))..., interval(mid(current_Y3)))))
        # Check if we can bisect
        if inf(f(current_Y1..., current_Y2..., current_Y3)) <= upper_bound
            if(diam(current_Y1)>= diam(current_Y2) && diam(current_Y1)>= diam(current_Y3))
                if(diam(current_Y1)>=0.0001)
                    S = bisect(current_Y1)
                    push!(working_list, (S[1],current_Y2,current_Y3), (S[2],current_Y2,current_Y3))
                else
                    push!(working_list,(current_Y1,current_Y2))
                end
            elseif(diam(current_Y2)>= diam(current_Y1) && diam(current_Y2)>= diam(current_Y3))
                if(diam(current_Y2)>=0.0001)
                    S1 = bisect(current_Y2)
                    push!(working_list, (current_Y1,S1[1],current_Y3), (current_Y1,S1[2],current_Y3))
                else
                    push!(working_list,(current_Y1,current_Y2))
                end
            else
                if(diam(current_Y3)>=0.0001)
                    S2 = bisect(current_Y3)
                    push!(working_list, (current_Y1,current_Y2,S2[1]), (current_Y1,current_Y2,S2[2]))
                else
                    push!(working_list,(current_Y1,current_Y2))
                end
            end
        end

        # Store the current intervals and upper bounds
        #push!(interval_lists, copy(working_list))
        push!(upper_bounds, upper_bound)
    end

    return working_list, upper_bounds
end



calculate_branch_bound (generic function with 1 method)

In [ ]:

# # Define the branch-and-bound function
# function calculate_branch_bound(f, Y1::Interval{T}, Y2::Interval{T}, Y3::Interval{T}) where T
#     interval_lists = [[Y1, Y2, Y3]]
#     upper_bound = sup(f(Y1..., Y2..., Y3...))
#     upper_bounds = [upper_bound]
#         # Update the upper bound
#         upper_bound = min(upper_bound, sup(f(Y1..., Y2..., Y3)))
#         # Check if we can bisect
#         if inf(f(Y1..., Y2..., Y3)) <= upper_bound
#             if(diam(Y1)>= diam(Y2) && diam(Y1)>= diam(Y3))
#                 if(diam(Y1)>=0.006)
#                     S = bisect(Y1)
#                     calculate_branch_bound(calculate_expression,S[1],Y2,Y3)
#                     calculate_branch_bound(calculate_expression,S[2],Y2,Y3)
#                     #push!(working_list, (S[1],current_Y2,current_Y3), (S[2],current_Y2,current_Y3))
#                 else
#                     push!(working_list,(Y1,Y2,Y3))
#                     break
#                 end
#             elseif(diam(Y2)>= diam(Y1) && diam(Y2)>= diam(Y3))
#                 if(diam(Y2)>=0.006)
#                     S1 = bisect(Y2)
#                     calculate_branch_bound(calculate_expression,Y1,S1[1],Y3)
#                     calculate_branch_bound(calculate_expression,Y1,S1[2],Y3)
#                     #push!(working_list, (current_Y1,S1[1],current_Y3), (current_Y1,S1[2],current_Y3))
#                 else
#                     push!(working_list,(Y1,Y2,Y3))
#                     break
#                 end
#             elseif(diam(Y3)>=0.006)
#                     S2 = bisect(Y3)
#                     calculate_branch_bound(calculate_expression,Y1,Y2,S2[1])
#                     calculate_branch_bound(calculate_expression,Y1,Y2,S2[2])
#                     #push!(working_list, (current_Y1,current_Y2,S2[1]), (current_Y1,current_Y2,S2[2]))
#                 else
#                     push!(working_list,(Y1,Y2,Y3))
#                     break
#                 end
#             end
#         end

#         # Store the current intervals and upper bounds
#         #push!(interval_lists, copy(working_list))
#         push!(upper_bounds, upper_bound)
#     end

#     return working_list, upper_bounds
# end



In [ ]:
function calculate_branch_bound(f, Y1::Interval{T}, Y2::Interval{T}, Y3::Interval{T}) where T
    interval_lists = [[Y1, Y2, Y3]]
    upper_bound = sup(f(Y1..., Y2..., Y3...))
    upper_bounds = [upper_bound]
    working_list = [(Y1..., Y2..., Y3...)]

    while !isempty(working_list)
        (current_Y1, current_Y2, current_Y3) = popfirst!(working_list)
        upper_bound = min(upper_bound, sup(f(current_Y1..., current_Y2..., current_Y3...)))
        if inf(f(current_Y1..., current_Y2..., current_Y3...)) <= upper_bound
            if diam(current_Y1) <= 0.0004 && diam(current_Y2) <= 0.0004 && diam(current_Y3) <= 0.0004
                push!(working_list, (current_Y1, current_Y2, current_Y3))
                push!(upper_bounds, upper_bound)
                break 
            elseif (diam(current_Y1)>= diam(current_Y2) && diam(current_Y1)>= diam(current_Y3)) && diam(current_Y1) >= 0.0004
                S = bisect(current_Y1)
                push!(working_list, (S[1], current_Y2, current_Y3), (S[2], current_Y2, current_Y3))
            elseif (diam(current_Y2)>= diam(current_Y1)) && diam(current_Y2)>= diam(current_Y3) && diam(current_Y2) >= 0.0004
                S1 = bisect(current_Y2)
                push!(working_list, (current_Y1, S1[1], current_Y3), (current_Y1, S1[2], current_Y3))
            elseif diam(current_Y3) >= 0.0004
                S2 = bisect(current_Y3)
                push!(working_list, (current_Y1, current_Y2, S2[1]), (current_Y1, current_Y2, S2[2]))
            else
                push!(working_list, (current_Y1, current_Y2, current_Y3))
            end
        end

        # Store the current intervals and upper bounds
        push!(upper_bounds, upper_bound)
    end

    return working_list, upper_bounds
end

In [4]:
N = 200000
y1_interval = interval(0.0, 1)
y2_interval = interval(0.0, 1)
y3_interval = interval(0.0, 1)
working_list, minimas = calculate_branch_bound(calculate_expression, y1_interval, y2_interval,y3_interval, N)
#working_list, minimas = calculate_branch_bound(calculate_expression, y1_interval, y2_interval,y3_interval)

(Tuple{Interval{Float64}, Interval{Float64}, Interval{Float64}}[([0.404296, 0.406251]_com, [0.234374, 0.238282]_com, [0.0976562, 0.101563]_com), ([0.402343, 0.404297]_com, [0.238281, 0.242188]_com, [0.0937499, 0.0976563]_com), ([0.404296, 0.406251]_com, [0.238281, 0.242188]_com, [0.0937499, 0.0976563]_com), ([0.402343, 0.404297]_com, [0.238281, 0.242188]_com, [0.0976562, 0.101563]_com), ([0.404296, 0.406251]_com, [0.238281, 0.242188]_com, [0.0976562, 0.101563]_com), ([0.398437, 0.400391]_com, [0.234374, 0.238282]_com, [0.101562, 0.105469]_com), ([0.40039, 0.402344]_com, [0.234374, 0.238282]_com, [0.101562, 0.105469]_com), ([0.398437, 0.400391]_com, [0.234374, 0.238282]_com, [0.105468, 0.109376]_com), ([0.40039, 0.402344]_com, [0.234374, 0.238282]_com, [0.105468, 0.109376]_com), ([0.398437, 0.400391]_com, [0.238281, 0.242188]_com, [0.101562, 0.105469]_com)  …  ([0.398437, 0.400391]_com, [0.238281, 0.240235]_com, [0.0976562, 0.101563]_com), ([0.398437, 0.400391]_com, [0.240234, 0.242188]

In [37]:
begin
    z = calculate_expression(interval((0.402343, 0.404297)),interval((0.236328, 0.238282)),interval((0.0976562, 0.101563)))
    println(z)
end

[0.592065, 0.595677]_com_NG
